# Online News Popularity - Veri Ön İşleme

Bu notebook aşağıdaki işlemleri gerçekleştirir:
1. `dataset.csv` dosyasını okuma
2. Ayırt edici bilgi içermeyen özelliklerin (url, timedelta) çıkarılması
3. Hedef değişkenin (shares) binary sınıflandırmaya dönüştürülmesi (eşik: 1400)

In [2]:
import pandas as pd
import numpy as np
import os

## 1. veri kümesini Okuma

In [3]:
# Dataset yolunu belirle
dataset_dir = os.path.join("..", "dataset_files")
dataset_path = os.path.join(dataset_dir, 'dataset.csv')

# veri kümesini oku
df = pd.read_csv(dataset_path)

print(f"veri kümesi boyutu: {df.shape}")
print(f"Toplam özellik sayısı: {df.shape[1]}")
print(f"Toplam örnek sayısı: {df.shape[0]}")
df.head()

veri kümesi boyutu: (39644, 61)
Toplam özellik sayısı: 61
Toplam örnek sayısı: 39644


,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [4]:
# Sütun isimlerini göster
print("Sütunlar:")
print(df.columns.tolist())

Sütunlar:
['url', ' timedelta', ' n_tokens_title', ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares', ' self_reference_avg_sharess', ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04', ' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words', ' global_rate_negative_words', ' rate_positive

## 2. Ayırt Edici Bilgi İçermeyen Özellikleri Çıkarma

- **url**: Makale URL'si - tanımlayıcı bilgi, tahmin için kullanılamaz
- **timedelta**: Makale yayın tarihi ile veri kümesi toplama tarihi arasındaki gün sayısı - ayırt edici değil

In [5]:
# Çıkarılacak sütunları belirle
columns_to_remove = ['url', ' timedelta']  # CSV'deki sütun isimlerinde boşluk olabilir

# Sütun isimlerini temizle (baştaki ve sondaki boşlukları kaldır)
df.columns = df.columns.str.strip()

# Şimdi çıkarılacak sütunları güncelle
columns_to_remove = ['url', 'timedelta']

# Mevcut sütunları kontrol et ve sadece var olanları çıkar
existing_columns_to_remove = [col for col in columns_to_remove if col in df.columns]
print(f"Çıkarılacak sütunlar: {existing_columns_to_remove}")

# Sütunları çıkar
df_processed = df.drop(columns=existing_columns_to_remove)

print(f"\nÇıkarma sonrası veri kümesi boyutu: {df_processed.shape}")
print(f"Kalan özellik sayısı: {df_processed.shape[1]}")

Çıkarılacak sütunlar: ['url', 'timedelta']

Çıkarma sonrası veri kümesi boyutu: (39644, 59)
Kalan özellik sayısı: 59


## 3. Hedef Değişkeni Binary Sınıflandırmaya Dönüştürme

Shares (paylaşım sayısı) hedef değişkeni için:
- **Eşik değeri**: 1400 (medyan değer)
- shares >= 1400 → **is_popular = 1** (popüler)
- shares < 1400 → **is_popular = 0** (popüler değil)

In [6]:
# Shares sütununun orijinal dağılımını göster
print("Shares sütunu istatistikleri:")
print(df_processed['shares'].describe())
print(f"\nMedyan değer: {df_processed['shares'].median()}")

Shares sütunu istatistikleri:
count     39644.000000
mean       3395.380184
std       11626.950749
min           1.000000
25%         946.000000
50%        1400.000000
75%        2800.000000
max      843300.000000
Name: shares, dtype: float64

Medyan değer: 1400.0


In [7]:
# Eşik değeri
THRESHOLD = 1400

# Binary sınıf oluştur: shares >= 1400 ise 1, değilse 0
df_processed['is_popular'] = (df_processed['shares'] >= THRESHOLD).astype(int)

# Sınıf dağılımını göster
print("Sınıf dağılımı:")
print(df_processed['is_popular'].value_counts())
print(f"\nSınıf 0 (shares < {THRESHOLD}): {(df_processed['is_popular'] == 0).sum()} örnek")
print(f"Sınıf 1 (shares >= {THRESHOLD}): {(df_processed['is_popular'] == 1).sum()} örnek")

Sınıf dağılımı:
is_popular
1    21154
0    18490
Name: count, dtype: int64

Sınıf 0 (shares < 1400): 18490 örnek
Sınıf 1 (shares >= 1400): 21154 örnek


In [8]:
# Shares sütununu kaldır (artık is_popular sütunumuz var)
df_final = df_processed.drop(columns=['shares'])

print(f"Son veri kümesi boyutu: {df_final.shape}")
print(f"\nSon sütunlar:")
print(df_final.columns.tolist())

Son veri kümesi boyutu: (39644, 59)

Son sütunlar:
['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length', 'num_keywords', 'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words', 'rate_positive_words', 'rate_negative_wor

## 4. Sonuç ve Özet

In [9]:
print("=" * 50)
print("VERİ ÖN İŞLEME ÖZETİ")
print("=" * 50)
print(f"Orijinal veri kümesi boyutu: {df.shape}")
print(f"İşlenmiş veri kümesi boyutu: {df_final.shape}")
print(f"\nÇıkarılan özellikler: url, timedelta")
print(f"Eklenen özellik: is_popular (binary hedef değişken)")
print(f"\nEşik değeri: {THRESHOLD}")
print(f"Sınıf 0 oranı: {(df_final['is_popular'] == 0).mean():.2%}")
print(f"Sınıf 1 oranı: {(df_final['is_popular'] == 1).mean():.2%}")
print("=" * 50)

# Son veri kümesini göster
df_final.head()

VERİ ÖN İŞLEME ÖZETİ
Orijinal veri kümesi boyutu: (39644, 61)
İşlenmiş veri kümesi boyutu: (39644, 59)

Çıkarılan özellikler: url, timedelta
Eklenen özellik: is_popular (binary hedef değişken)

Eşik değeri: 1400
Sınıf 0 oranı: 46.64%
Sınıf 1 oranı: 53.36%


,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,is_popular
0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,0
1,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,0
2,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1
3,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,0
4,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,0


In [10]:
# İşlenmiş veri kümesini kaydet (isteğe bağlı)
output_path = os.path.join(dataset_dir, "processed_dataset.csv")
df_final.to_csv(output_path, index=False)
print(f"İşlenmiş veri kümesi kaydedildi: {output_path}")

İşlenmiş veri kümesi kaydedildi: ..\dataset_files\processed_dataset.csv
